In [1]:
import os
from surfer import Brain, project_volume_data
from mayavi import mlab
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw

mlab.init_notebook(backend='png')

Notebook initialized with png backend.


In [4]:
def proj_surf(file_name, colorbar = None):
    ROOT_DIR = "/Users/vgonzenb/PennSIVE/ISLA/"
    MRI_PATH = os.path.join("data", "mri", "reg")

    mri_file = os.path.join(ROOT_DIR + MRI_PATH, file_name)
    reg_file = os.path.join(os.environ["FREESURFER_HOME"], "average/mni152.register.dat")
    
    """
    Project volume to surface
    """
    brain = Brain(subject_id = "fsaverage5", hemi = "split", surf = "inflated",
              views=['med', 'lat'], background="white", size=1200)

    surf_data_lh = project_volume_data(mri_file, "lh", reg_file, smooth_fwhm=3)
    surf_data_rh = project_volume_data(mri_file, "rh", reg_file, smooth_fwhm=3)

    brain.add_overlay(surf_data_lh, min=0, max=90, name="cbf_lh", hemi='lh', sign="pos")
    brain.add_overlay(surf_data_rh, min=0, max=90, name="cbf_rh", hemi='rh', sign="pos")

    brain.save_imageset(prefix="tmp/rh", views=["med", "lat"], colorbar=colorbar)
    brain.save_imageset(prefix="tmp/lh", views=["med", "lat"], row=-2, col=-2, colorbar=None)

    """
    Contenate surface images horizontally
    """
    im1 = Image.open(os.path.join(os.getcwd(), 'tmp/lh_med.png'))
    im2 = Image.open(os.path.join(os.getcwd(), 'tmp/lh_lat.png'))
    im3 = Image.open(os.path.join(os.getcwd(), 'tmp/rh_lat.png'))
    im4 = Image.open(os.path.join(os.getcwd(), 'tmp/rh_med.png'))
    
    img = Image.new('RGB', (im1.width * 4, im1.height))
    img.paste(im1, (0, 0))
    img.paste(im2, (im1.width, 0))
    img.paste(im3, (im1.width*2, 0))
    img.paste(im4, (im1.width*3, 0))
    
    """
    Add text to each image
    """

    draw = ImageDraw.Draw(img)
    # font = ImageFont.truetype(<font-file>, <font-size>)
    font = ImageFont.truetype("/Library/Fonts/HelveticaNeue.ttc", 50)
    font_BOLD = ImageFont.truetype("/Library/Fonts/HelveticaNeue.ttc", 100)

    # draw.text((x, y),"Sample Text",(r,g,b))
    draw.text((im1.width-25, im1.height-525), "Left", (0,0,0), font=font)
    draw.text((im1.width*3-70, im1.height-525), "Right", (0,0,0), font=font)

    clean_filename = file_name.replace('.nii.gz', '')
    img.save(f'tmp/pub_{clean_filename}.png')
    print(f"Image saved on 'tmp/pub_{clean_filename}.png")
    return(None)

In [5]:
proj_surf("avgAhlgren_cbf_size3_thr10.nii.gz")
proj_surf("avgISLA_cbf_size3_thr10.nii.gz")
proj_surf("avgCBF_thr10.nii.gz")

Image saved on 'tmp/pub_avgAhlgren_cbf_size3_thr10.png
Image saved on 'tmp/pub_avgISLA_cbf_size3_thr10.png
Image saved on 'tmp/pub_avgCBF_thr10.png


Unused Below